In [1]:
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of the text: {} characters'.format(len(text)))

Length of the text: 1115394 characters


In [10]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

In [5]:
text_as_int = text_to_int(text)

In [13]:
print('{} ---- characters mapped to int ---- > {}'.format(text[:13], text_as_int[:13]))

First Citizen ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [6]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [15]:
print(int_to_text(text_as_int[:13]))

First Citizen


In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
for x, y in dataset.take(2):
  print('\n\n Example \n')
  print("INPUT")
  print(int_to_text(x))
  print("TARGET")
  print(int_to_text(y))



 Example 

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
TARGET
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


 Example 

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 
TARGET
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [10]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # number of unique chraracters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# buffer size to shuffle the data
# ( tf data is designed to work with possibly infinte sequences, so it doesn't attempt to shuffle the entire sequence in memory. Instead it maintains a buffer in memory and shuffles the buffer)

BUFFER_SIZE = 10000

data = dataset.shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [11]:
def build_model(vocab_size, embedding_dim, rnn_unit, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_unit, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_prediction = model(input_example_batch)
  print(example_batch_prediction.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
print(len(example_batch_prediction))
print(example_batch_prediction)

64
tf.Tensor(
[[[-3.6949608e-03  2.7353715e-04  3.2527572e-03 ... -2.8428072e-03
    8.8533526e-04 -1.7859146e-04]
  [ 5.4476303e-03 -4.1701952e-03 -2.3412150e-03 ...  2.2141607e-03
    7.7489377e-03  9.2974445e-04]
  [ 1.3644922e-02 -1.9484917e-03 -1.0778324e-03 ... -2.0351270e-03
    7.3579066e-03  6.8059461e-03]
  ...
  [ 5.9976541e-03 -2.0526990e-03 -4.9554147e-03 ... -8.0194732e-04
   -3.7999655e-04 -1.4247971e-03]
  [ 8.5791312e-03 -1.8906562e-03  3.0837785e-03 ... -4.5740046e-03
    2.8129914e-03 -3.7806341e-04]
  [ 2.8496897e-03 -1.0516122e-04 -2.0917254e-03 ... -3.4135520e-03
    4.9899817e-03 -4.3369168e-03]]

 [[ 3.1667808e-03 -1.1396809e-03  6.9900369e-03 ... -2.8581582e-03
    3.1413180e-03  1.1248235e-03]
  [-2.2382303e-03  3.3094781e-05  9.3078637e-04 ... -1.2569681e-03
    5.4983636e-03 -2.8896974e-03]
  [-1.0223915e-02  5.6589316e-03  4.4127153e-03 ... -3.5803139e-03
   -3.1360688e-03 -3.3164080e-03]
  ...
  [ 1.1809905e-02 -1.7841556e-03 -9.2433468e-03 ...  4.0204655e

In [15]:
# Creating a loss function for 3d array
pred = example_batch_prediction[0]

sampled_indices = tf.random.categorical(pred, num_samples=1)
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"Vx?HqLveHCd\n'V?Ns LBsDN.MxjOP?hi?X gCTkuqs'mW?e;ooNOxcV?rQBYuJRz3LExrwJ,!qDL,OmjDGXTmfyms.EzBTCllQq'"

In [16]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [49]:
model.compile(optimizer='adam', loss=loss)

In [17]:
# checkpoint
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath = checkpoint_prefix,
  save_weights_only = True,
  save_format = 'hdf5'
)

In [17]:
# training
history = model.fit(data, epochs=2, callbacks=[checkpoint_callback])

Epoch 1/2
172/172 [==============================] - 977s 6s/step - loss: 2.5638
Epoch 2/2
172/172 [==============================] - 915s 5s/step - loss: 1.8642


In [18]:
model.save('shakespeare.h5')

In [18]:
new_model = tf.keras.models.load_model('shakespare.h5', custom_objects={'loss': loss})

In [19]:
new_model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [20]:
new_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
new_model.build(tf.TensorShape([1, None]))

In [21]:
checkpoint_num = 2
new_model.load_weights(checkpoint_prefix.format(epoch=checkpoint_num))

In [22]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # low temperatures results in more predictable text.
  # higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.6

  # Here batch size == 1
  new_model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state

    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [23]:
inp = input('Type a starting string: ')
print(generate_text(new_model, inp))

romeow the knomes comntt there all best to mises, the seall dopress for my wishen,
The follor saring your should some to the and cold hill the sirest for thay my sould meath of the pray;
What the starest for beather,
And by woll have for why srucaing bestinger in heart speather hear and to to the greal of the lody sterth not be the feather sof the death of here you rearsul there and reand to sto,
When it the dines cain streeth pad the what bake he treest to cam stand tis bendest and made a not ane tood hither mare
That not hear me in the proped and have all asterfower your warder the maret if thou dortay to wise to miser as of I besere to the song the

AUTIO:
Yit hath so hear the have mongh co thy loves of of here and the forsief wor her:
The with for the come in sees the in to purest of the it com hour of herred being ther hank thou to thou dece deart.

Prister:
And with speer caksed her were of uppince the wall here om the sope,
The will ta main to thou serel came of seeting my may y